In [9]:
import torch
from torch import nn, tensor, mean, unsqueeze, cat, transpose, matmul, sigmoid, arange, abs, optim, zeros
from torch.optim.lr_scheduler import LambdaLR
import torch.nn.functional as F
from torch.autograd import Variable
from nltk.tokenize import word_tokenize, sent_tokenize
from numpy import random
import numpy as np
import re
import pickle
import os
import json

In [3]:
with open("reviews_Movies_and_TV.json") as json_file:
    file_contents = json_file.readlines()
    
no_of_reviews = len(file_contents)

sentences = []
words = []
word_freq = {}
count = 0
MAX_SENTENCES = 40000

for i in range(no_of_reviews):
    review = json.loads(file_contents[i])['reviewText']
    review_sent = sent_tokenize(review.lower())
    review = re.sub(r'(?<=[a-zA-Z])(\.\?\!)(?=[a-zA-Z])', '\1 ', review)
#     for i,review in enumerate(review_sent):
#         review_sent[i] = re.sub(r'[\.\?\!]', '', review)
    sentences += review_sent
    count += len(review_sent)
    for sent in review_sent:
        review_words = word_tokenize(sent)
        words += review_words
        for word in review_words:
            try: word_freq[word] += 1
            except KeyError: word_freq[word] = 1
    if count >= MAX_SENTENCES:
        break
        
freq_threshold = 3
words = [word if word_freq[word] > freq_threshold else '<unk>' for word in words]

sentence_list = sentences.copy()
for i,sentence in enumerate(sentence_list):
    ws = word_tokenize(sentence)
    ws = [w if word_freq[w] > freq_threshold else '<unk>' for w in ws]
    sentences[i] = ws

In [4]:
word_freq = {}
for word in words:
    try: word_freq[word] += 1
    except KeyError: word_freq[word] = 0
        
vocab = list(word_freq.keys())

index_to_word = {i: list(vocab)[i] for i in range(len(vocab)) }
word_to_index = {word: i for i, word in index_to_word.items()}

In [ ]:
def get_dataset(self):
    xTrain = []
    yTrain = []
    c = 0
    with open("tokenised.txt", "r") as f:
        self.distribution = [0]*len(self.vocab)
        total_words = 0
        for line in f:
            c += 1
            words = eval(line)
            total_words += len(words)
            for word in words:
                self.distribution[self.word_indices[word]] += 1

            if (c % 1000 == 0):
                print("line number", c)
            if (c == STOP_AT):
                break

        self.distribution = list(map(lambda x: x/total_words,
                                     self.distribution))

    print("Getting data")
    with open("tokenised.txt", "r") as f:
        #one_hots = F.one_hot(arange(len(self.vocab))).float()
        z = zeros(len(self.vocab))
        def one_hot(index):
            x = z
            x[index] = 1.
            return x

        c = 0
        for line in f:
            c += 1
            words = eval(line)

            window = words[:self.window_size*2+1]
            for t, word in enumerate \
                             (words[self.window_size:-self.window_size]):
                context = window.copy()
                context.pop(self.window_size)
                #context = words[t:t+self.window_size] \
                #        + words[t+self.window_size+1:t+self.window_size*2+1]
                try: window = window[1:] + [words[t + self.window_size*2+1]]
                except: pass
                xTrain.append(cat
                                (list(map(lambda w: unsqueeze(one_hot(self.word_indices[w]),0),
                                           context +
                                           [word] +
                                           self.get_noise(context, word)))))
                yTrain.append(tensor([1.] + [0.]*self.noise))

            #if (c % 1000 == 0):
            print("line number", c)
            if (c == STOP_AT):
                break

    self.trainset = ([t.float() for t in xTrain], [t.float() for t in yTrain])}